In [ ]:
using Intan, PyPlot

In [ ]:
#First create amplifier array:
myamp=RHD2164("PortA1")

#Task
mt=Task_TestTask()

#Save
mys=SaveNone()

#Initialize evaluation board setup
myrhd=makeRHD(myamp,"single",mt,sav=mys);

In [ ]:
handles = makegui(myrhd);

In [ ]:
function get_times(rhd,streams,samples,sr::Int64)
    
    myt=zeros(Float64,length(streams),length(samples))
    mytest=zeros(Float64,5)
    for s=1:length(streams)
        for j=1:length(samples)
            thebytes=(4+2(streams[s]*36)+8+2)*2*samples[j]
            for k=1:length(mytest)
                mytest[k]=@elapsed Intan.ReadFromPipeOut(rhd,Intan.PipeOutData,convert(Clong,thebytes),rhd.usbBuffer)   
            end
            myt[s,j]=mean(mytest)/(samples[j]/sr)
        end
    end
    
    myt.*100
end

In [ ]:
mysamples=collect(50:50:600)
sample_rate=20000
mystreams=collect(1:16)
t=get_times(myrhd,mystreams,mysamples,sample_rate);

In [ ]:
for i=1:size(t,1)
    plot(mysamples./sample_rate.*1000,t[i,:]',color="blue",alpha=i/20)
end

xlabel("Latency (ms)")
ylabel("% of processing window")

In [ ]:
for i=1:size(t,2)
    plot(collect(32:32:512),t[:,i],color="blue", alpha=i/20)
end
xlabel("Number of channels")
ylabel("% of Processing Window")